In [37]:
import csv
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import sklearn

In [40]:
def process_image(path):
    filename = path.split('/')[-1]
    current_path = base_path + 'IMG/' + filename
    return cv2.imread(current_path)
    
def flip(image, measurement):
    return np.fliplr(image), -measurement

images = []
measurements = []
for line in lines[1:]:
    center_image = process_image(line[0])
    left_image = process_image(line[1])
    right_image = process_image(line[2])

    correction = 0.2
    center_measurement = float(line[3])
    left_measurement = center_measurement + correction
    right_measurement = center_measurement - correction
    
    # center
    images.append(center_image)
    measurements.append(center_measurement)
    iflip, mflip = flip(center_image, center_measurement)
    images.append(iflip)
    measurements.append(mflip)    
    
    # left
    images.append(left_image)
    measurements.append(left_measurement)
    iflip, mflip = flip(left_image, left_measurement)
    images.append(iflip)
    measurements.append(mflip)    
    
    # right
    images.append(right_image)
    measurements.append(right_measurement)
    iflip, mflip = flip(right_image, right_measurement)
    images.append(iflip)
    measurements.append(mflip)    

In [41]:
len(images)

20760

In [42]:
X_train = np.array(images)
y_train = np.array(measurements)

In [43]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Convolution2D, MaxPooling2D, Cropping2D

In [103]:
base_path = '/hdd/datasets/udacity_sim_data/recorded_data/'
# base_path = '/hdd/datasets/udacity_sim_data/recorded_data/train/'
# base_path = '/hdd/datasets/udacity_sim_data/data/'

In [104]:
import os
import csv

samples = []
with open(base_path + 'train/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
with open(base_path + 'recovery/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

In [117]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

In [118]:
correction = 0.3

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
#                 name = base_path + '/IMG/'+batch_sample[0].split('/')[-1]
                name = batch_sample[0]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                images.append(np.fliplr(center_image))
                angles.append(-center_angle)
                
#                 name = base_path + '/IMG/'+batch_sample[1].split('/')[-1]
                name = batch_sample[1]
                left_image = cv2.imread(name)
                left_angle = center_angle + correction
                images.append(left_image)
                angles.append(left_angle)
                images.append(np.fliplr(left_image))
                angles.append(-left_angle)

#                 name = base_path + '/IMG/'+batch_sample[2].split('/')[-1] 
                name = batch_sample[2]
                right_image = cv2.imread(name)
                right_angle = center_angle - correction
                images.append(right_image)
                angles.append(right_angle)
                images.append(np.fliplr(right_image))
                angles.append(-right_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [119]:
input_shape=(160,320,3)

model = Sequential()
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=input_shape))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [120]:
model.compile(loss='mse', optimizer='adam')

In [121]:
model.fit_generator(train_generator, 
                    samples_per_epoch= len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), 
                    nb_epoch=6)
model.save('model.h5')

Epoch 1/6
3072/3212 [===========================>..] - ETA: 0s - loss: 0.0688

/home/ryan/.conda/envs/dl35/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3264/3212 [==============================] - 6s - loss: 0.0688 - val_loss: 0.0593
Epoch 2/6
3264/3212 [==============================] - 4s - loss: 0.0520 - val_loss: 0.0435
Epoch 3/6
3264/3212 [==============================] - 4s - loss: 0.0424 - val_loss: 0.0425
Epoch 4/6
3264/3212 [==============================] - 4s - loss: 0.0392 - val_loss: 0.0438
Epoch 5/6
3264/3212 [==============================] - 4s - loss: 0.0389 - val_loss: 0.0404
Epoch 6/6
3336/3212 [===============================] - 4s - loss: 0.0342 - val_loss: 0.0390
